In [ ]:
import cdsapi

dataset = "reanalysis-era5-single-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": [
        "10m_u_component_of_wind",
        "10m_v_component_of_wind",
        "2m_temperature",
        "surface_pressure"
    ],
    "year": ["2018", "2019", "2020"],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "day": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12",
        "13", "14", "15",
        "16", "17", "18",
        "19", "20", "21",
        "22", "23", "24",
        "25", "26", "27",
        "28", "29", "30",
        "31"
    ],
    "time": [
        "00:00", "01:00", "02:00",
        "03:00", "04:00", "05:00",
        "06:00", "07:00", "08:00",
        "09:00", "10:00", "11:00",
        "12:00", "13:00", "14:00",
        "15:00", "16:00", "17:00",
        "18:00", "19:00", "20:00",
        "21:00", "22:00", "23:00"
    ],
    "data_format": "netcdf",                # maybe change data_format to format ?
    "download_format": "unarchived",
    "area": [35.5, -118.8, 34.7, -117.8]
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()


HTTPError: 403 Client Error: Forbidden for url: https://cds.climate.copernicus.eu/api/retrieve/v1/processes/reanalysis-era5-single-levels/execution
cost limits exceeded
Your request is too large, please reduce your selection.

In [11]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    "reanalysis-era5-single-levels",
    {
        "product_type": "reanalysis",
        "format": "netcdf",
        "variable": [
            "10m_u_component_of_wind",
            "10m_v_component_of_wind",
            "2m_temperature",
            "surface_pressure",
        ],
        "year": ["2018", "2019", "2020"],
        "month": [f"{m:02d}" for m in range(1, 13)],
        "day": [f"{d:02d}" for d in range(1, 32)],
        "time": [f"{h:02d}:00" for h in range(0, 24)],
        # small box around Tehachapi: [North, West, South, East]
        "area": [35.5, -118.8, 34.7, -117.8],
    },
    "era5_tehachapi_2018_2020.nc",
)


HTTPError: 403 Client Error: Forbidden for url: https://cds.climate.copernicus.eu/api/retrieve/v1/processes/reanalysis-era5-single-levels/execution
cost limits exceeded
Your request is too large, please reduce your selection.

In [10]:
# Quick inspection of the .nc file
import xarray as xr

ds = xr.open_dataset("era5_tehachapi_2018_2020.nc", engine="h5netcdf")
print(ds)


OSError: Unable to synchronously open file (file signature not found)

In [ ]:
# Quick sanity checks
print(ds.time.values[0], ds.time.values[-1])
print(ds.u10.shape)
print(ds.u10.mean().item(), ds.v10.mean().item())

In [ ]:
# convert to dataframe
df = ds.to_dataframe().reset_index()
df.head()

In [ ]:
import matplotlib.pyplot as plt

df.set_index("time")[["u10", "v10"]].plot(figsize=(10,3))
plt.title("ERA5 10 m wind components at Tehachapi (2018-2020)")
plt.show()


In [ ]:
import numpy as np
df["w"] = np.sqrt(df["u10"]**2 + df["v10"]**2)
print(df["w"].describe())
